In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crowdflower-search-relevance/train.csv.zip
/kaggle/input/crowdflower-search-relevance/sampleSubmission.csv.zip
/kaggle/input/crowdflower-search-relevance/test.csv.zip


In [80]:
train = pd.read_csv('/kaggle/input/crowdflower-search-relevance/train.csv.zip')
test = pd.read_csv('/kaggle/input/crowdflower-search-relevance/test.csv.zip')
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [81]:
from bs4 import BeautifulSoup
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()


train['query'] = train['query'].apply(remove_html_tags)
test['query'] = test['query'].apply(remove_html_tags)

In [82]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10158 entries, 0 to 10157
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10158 non-null  int64  
 1   query                10158 non-null  object 
 2   product_title        10158 non-null  object 
 3   product_description  7714 non-null   object 
 4   median_relevance     10158 non-null  int64  
 5   relevance_variance   10158 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 476.3+ KB


In [83]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22513 entries, 0 to 22512
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   22513 non-null  int64 
 1   query                22513 non-null  object
 2   product_title        22513 non-null  object
 3   product_description  17086 non-null  object
dtypes: int64(1), object(3)
memory usage: 703.7+ KB


In [84]:
train.describe()

,id,median_relevance,relevance_variance
count,10158.000000,10158.000000,10158.000000
mean,16353.103071,3.309805,0.377863
std,9447.106683,0.980666,0.389707
min,1.000000,1.000000,0.000000
25%,8078.750000,3.000000,0.000000
50%,16349.500000,4.000000,0.471000
75%,24570.750000,4.000000,0.471000
max,32668.000000,4.000000,1.470000


In [85]:
train['query'].map(lambda x:len(x.split())).value_counts()

2    5379
3    2819
4     925
1     885
6      81
5      69
Name: query, dtype: int64

In [86]:
train['product_title'].map(lambda x:len(x.split())).value_counts()

7     1288
6     1284
8     1183
9     1122
5     1002
10     880
11     744
12     678
4      550
13     453
14     289
15     181
3      174
17      78
16      66
2       51
18      25
19      24
20      15
24      11
21       7
25       7
27       6
26       6
22       6
28       6
23       6
29       4
1        2
46       2
32       2
38       1
44       1
34       1
31       1
43       1
41       1
Name: product_title, dtype: int64

Feature engineering before vector

In [87]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction import text
import string

# Initialize the Porter Stemmer
stemmer = PorterStemmer()

# Stopwords and Punctuation
stop_words = ['http', 'www', 'img', 'border', 'color', 'style', 'padding', 'table', 'font', 'thi', 'inch', 'ha', 'width', 'height',
              '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
stop_words = text.ENGLISH_STOP_WORDS.union(stop_words)

punct = string.punctuation
punct_re = re.compile('[{}]'.format(re.escape(punct)))

columns_to_preprocess = ['query', 'product_title', 'product_description']

def preprocess_text_column(df, column):
    df[column] = df[column].apply(lambda x: preprocess(str(x)))
    return df

def preprocess(x):
    x = x.lower()
    x = punct_re.sub(' ', x)
    new_x = []
    for token in x.split(' '):
        if token not in stop_words:
            new_x.append(stemmer.stem(token))
    return ' '.join(new_x)

# Example usage:
# Assuming you have 'train' and 'test' DataFrames
train = preprocess_text_column(train, 'query')
train = preprocess_text_column(train, 'product_title')
train = preprocess_text_column(train, 'product_description')

test = preprocess_text_column(test, 'query')
test = preprocess_text_column(test, 'product_title')
test = preprocess_text_column(test, 'product_description')


In [88]:
split = int(len(train)*0.8)
train_0, dev = train[:split], train[split:]

In [89]:
clean_train_1 = train_0[train_0.relevance_variance <1].copy()
clean_train_2 = train_0[train_0.relevance_variance <0.50].copy()
dev.describe()

,id,median_relevance,relevance_variance
count,2032.000000,2032.000000,2032.000000
mean,29406.014764,3.319390,0.361364
std,1870.217123,0.972218,0.379619
min,26215.000000,1.000000,0.000000
25%,27777.250000,3.000000,0.000000
50%,29410.000000,4.000000,0.471000
75%,31014.750000,4.000000,0.471000
max,32668.000000,4.000000,1.470000


In [90]:
train = clean_train_1
train_input = train.apply(lambda x: x['query']+' '+x['product_title'], axis=1)
dev_input =  dev.apply(lambda x: x['query']+' '+x['product_title'], axis=1)

In [91]:
clean_train_1.describe()

,id,median_relevance,relevance_variance
count,7558.000000,7558.000000,7558.000000
mean,13074.201111,3.344403,0.321038
std,7571.543134,0.974908,0.332482
min,1.000000,1.000000,0.000000
25%,6496.500000,3.000000,0.000000
50%,13129.000000,4.000000,0.471000
75%,19563.250000,4.000000,0.471000
max,26208.000000,4.000000,0.980000


In [92]:
clean_train_2.describe()

,id,median_relevance,relevance_variance
count,6206.000000,6206.000000,6206.000000
mean,13154.066549,3.432646,0.202590
std,7570.559783,0.959901,0.232434
min,1.000000,1.000000,0.000000
25%,6576.750000,3.000000,0.000000
50%,13299.500000,4.000000,0.000000
75%,19708.000000,4.000000,0.471000
max,26208.000000,4.000000,0.490000


In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(ngram_range=(1, 5),stop_words = 'english', strip_accents='unicode')
train_x = tfidf.fit_transform(train_input)
dev_x = tfidf.transform(dev_input)

In [94]:
print(train_x)

  (0, 109450)	0.19624310126460465
  (0, 9600)	0.19624310126460465
  (0, 45196)	0.19624310126460465
  (0, 131476)	0.19624310126460465
  (0, 25104)	0.17677693475696124
  (0, 70893)	0.19624310126460465
  (0, 109449)	0.19624310126460465
  (0, 9599)	0.19624310126460465
  (0, 45195)	0.19624310126460465
  (0, 131474)	0.17677693475696124
  (0, 25103)	0.17677693475696124
  (0, 46173)	0.19624310126460465
  (0, 70892)	0.19624310126460465
  (0, 109448)	0.19624310126460465
  (0, 9598)	0.19624310126460465
  (0, 45193)	0.17677693475696124
  (0, 131473)	0.17677693475696124
  (0, 25100)	0.13943206840666683
  (0, 119917)	0.16205134825726433
  (0, 46172)	0.19624310126460465
  (0, 70888)	0.18151751476490768
  (0, 109447)	0.19624310126460465
  (0, 9596)	0.17677693475696124
  (0, 45192)	0.17677693475696124
  (0, 131469)	0.13943206840666683
  :	:
  (7557, 58277)	0.1840052250601992
  (7557, 35956)	0.1759284798179473
  (7557, 89323)	0.1759284798179473
  (7557, 108549)	0.1701979378712004
  (7557, 108548)	0.1701

adding further feature engineering

In [95]:
print(train_input)

0       bridal shower decor accent pillow heart design...
1       led christma light set 10 batteri oper multi l...
2       projector viewson pro8200 dlp multimedia proje...
3       wine rack concept housewar wr 44526 solid wood...
4       light bulb wintergreen light christma led ligh...
                              ...                        
8120    acoust guitar clamp hot metal acoust electr gu...
8121    rain jacket urban republ boy  hood stripe rain...
8122    exten hardisk 500 gb 500gb extern hard drive u...
8123                 bike lock mighti amsterdam bike lock
8124    phillip coffe maker hamilton beach flexbrew wa...
Length: 7558, dtype: object


In [96]:
train_y, dev_y = train.median_relevance.to_list(), dev.median_relevance.to_list()
train_y = [(x-1)/3 for x in train_y]
dev_y = [(x-1)/3 for x in dev_y]
np.mean(train_y), np.max(train_y), np.min(train_y)

(0.781467760430449, 1.0, 0.0)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from scipy import sparse
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup
import re
from nltk.stem.porter import PorterStemmer
import string
from sklearn.feature_extraction import text


svd = TruncatedSVD(n_components=100)
train_x_tfidf = svd.fit_transform(train_x_tfidf)
dev_x_tfidf = svd.transform(dev_x_tfidf)

# Add new features
distances = []
quasi_jaccard = []

for row in train_x_tfidf:
    cos_distance = linear_kernel(row[:len(row)//2].reshape(1, -1), row[len(row)//2:].reshape(1, -1))[0][0]
    intersect = np.dot(row[:len(row)//2], row[len(row)//2:])
    union = np.sum(row[:len(row)//2]) + np.sum(row[len(row)//2:])
    quasi_jaccard.append(1.0 * intersect / union)
    distances.append(cos_distance)

new_features_train = np.column_stack([train_x_tfidf, np.array([distances, quasi_jaccard]).T])

distances = []
quasi_jaccard = []

for row in dev_x_tfidf:
    cos_distance = linear_kernel(row[:len(row)//2].reshape(1, -1), row[len(row)//2:].reshape(1, -1))[0][0]
    intersect = np.dot(row[:len(row)//2], row[len(row)//2:])
    union = np.sum(row[:len(row)//2]) + np.sum(row[len(row)//2:])
    quasi_jaccard.append(1.0 * intersect / union)
    distances.append(cos_distance)

new_features_dev = np.column_stack([dev_x_tfidf, np.array([distances, quasi_jaccard]).T])


In [97]:
'''import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from scipy import sparse
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup
import re
from nltk.stem.porter import PorterStemmer
import string
from sklearn.feature_extraction import text

# ... (previous imports and code)

class FeatureInserter():
    def __init__(self):
        pass

    def transform(self, X, y=None):
        distances = []
        quasi_jaccard = []

        for row in X.tocsr():
            row = row.toarray().ravel()

            if len(row.shape) == 1:
                row = row.reshape(1, -1)

            # Split the row into two parts
            part1, part2 = row[:, :row.shape[1]//2], row[:, row.shape[1]//2:]

            # Ensure both matrices have the same number of features
            min_features = min(part1.shape[1], part2.shape[1])

            # Check for zero denominator to avoid division by zero
            denominator = np.linalg.norm(part1[:, :min_features]) * np.linalg.norm(part2[:, :min_features])
            if denominator == 0:
                cos_distance = 0  # or any other suitable value
            else:
                cos_distance = 1.0 - np.dot(part1[:, :min_features], part2[:, :min_features].T) / denominator

            # Compute quasi-Jaccard similarity
            intersect = np.sum(np.minimum(part1[:, :min_features], part2[:, :min_features]))
            union = np.sum(np.maximum(part1[:, :min_features], part2[:, :min_features]))
            quasi_jaccard.append(1.0 * intersect / union)

            distances.append(cos_distance[0])

        return np.column_stack([X.toarray(), np.array([distances, quasi_jaccard]).T])

    def fit(self, X, y):
        return self

    def fit_transform(self, X, y, **fit_params):
        self.fit(X, y)
        return self.transform(X)

# Assuming train_input and dev_input are your raw text data
feature_inserter = FeatureInserter()
train_x_with_features = feature_inserter.fit_transform(train_input, train_y)
dev_x_with_features = feature_inserter.transform(dev_input)

# Assuming train_x and dev_x are your vectorized data
tfidf = TfidfVectorizer(ngram_range=(1, 5), stop_words='english', strip_accents='unicode')
train_x_tfidf = tfidf.fit_transform(train_input)
dev_x_tfidf = tfidf.transform(dev_input)

# Concatenate the new features with the TF-IDF vectorized data
train = np.column_stack([train_x_tfidf.toarray(), train_x_with_features])
dev = np.column_stack([dev_x_tfidf.toarray(), dev_x_with_features])'''

"import pandas as pd\nimport numpy as np\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.decomposition import TruncatedSVD\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.svm import SVC\nfrom sklearn.pipeline import Pipeline, FeatureUnion\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.model_selection import KFold\nfrom scipy import sparse\nfrom sklearn.metrics.pairwise import linear_kernel\nfrom bs4 import BeautifulSoup\nimport re\nfrom nltk.stem.porter import PorterStemmer\nimport string\nfrom sklearn.feature_extraction import text\n\n# ... (previous imports and code)\n\nclass FeatureInserter():\n    def __init__(self):\n        pass\n\n    def transform(self, X, y=None):\n        distances = []\n        quasi_jaccard = []\n\n        for row in X.tocsr():\n            row = row.toarray().ravel()\n\n            if len(row.shape) == 1:\n                row = row.reshape(1, -1)\n\n            # Split the row into two parts\n 

In [98]:
'''import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
#from sklearn.cross_validation import KFold
from scipy import sparse
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup
import re
from nltk.stem.porter import PorterStemmer
import string
from sklearn.feature_extraction import text

# ... (previous imports and code)

class FeatureInserter():
    def __init__(self):
        pass

    def transform(self, X, y=None):
        distances = []
        quasi_jaccard = []

        for row in X.tocsr():
            row = row.toarray().ravel()

            if len(row.shape) == 1:
                row = row.reshape(1, -1)

            # Split the row into two parts
            part1, part2 = row[:, :row.shape[1]//2], row[:, row.shape[1]//2:]

            # Ensure both matrices have the same number of features
            min_features = min(part1.shape[1], part2.shape[1])

            # Check for zero denominator to avoid division by zero
            denominator = np.linalg.norm(part1[:, :min_features]) * np.linalg.norm(part2[:, :min_features])
            if denominator == 0:
                cos_distance = 0  # or any other suitable value
            else:
                cos_distance = 1.0 - np.dot(part1[:, :min_features], part2[:, :min_features].T) / denominator

            # Compute quasi-Jaccard similarity
            intersect = np.sum(np.minimum(part1[:, :min_features], part2[:, :min_features]))
            union = np.sum(np.maximum(part1[:, :min_features], part2[:, :min_features]))
            quasi_jaccard.append(1.0 * intersect / union)

            distances.append(cos_distance[0])

        return np.column_stack([X.toarray(), np.array([distances, quasi_jaccard]).T])

    def fit(self, X, y):
        return self

    def fit_transform(self, X, y, **fit_params):
        self.fit(X, y)
        return self.transform(X)


# Example usage:
feature_inserter = FeatureInserter()

# Apply TF-IDF vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 5), stop_words='english', strip_accents='unicode')
train_x_tfidf = tfidf.fit_transform(train_input)
dev_x_tfidf = tfidf.transform(dev_input)

# Apply FeatureInserter
train_x = feature_inserter.fit_transform(train_x_tfidf, train_y)
dev_x = feature_inserter.transform(dev_x_tfidf)

'''

"import pandas as pd\nimport numpy as np\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.decomposition import TruncatedSVD\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.svm import SVC\nfrom sklearn.pipeline import Pipeline, FeatureUnion\nfrom sklearn.linear_model import LogisticRegression\n#from sklearn.cross_validation import KFold\nfrom scipy import sparse\nfrom sklearn.metrics.pairwise import linear_kernel\nfrom bs4 import BeautifulSoup\nimport re\nfrom nltk.stem.porter import PorterStemmer\nimport string\nfrom sklearn.feature_extraction import text\n\n# ... (previous imports and code)\n\nclass FeatureInserter():\n    def __init__(self):\n        pass\n\n    def transform(self, X, y=None):\n        distances = []\n        quasi_jaccard = []\n\n        for row in X.tocsr():\n            row = row.toarray().ravel()\n\n            if len(row.shape) == 1:\n                row = row.reshape(1, -1)\n\n            # Split the row into two parts\

In [99]:
'''train_y, dev_y = train.median_relevance.to_list(), dev.median_relevance.to_list()
train_y = [(x-1)/3 for x in train_y]
dev_y = [(x-1)/3 for x in dev_y]
np.mean(train_y), np.max(train_y), np.min(train_y)'''

'train_y, dev_y = train.median_relevance.to_list(), dev.median_relevance.to_list()\ntrain_y = [(x-1)/3 for x in train_y]\ndev_y = [(x-1)/3 for x in dev_y]\nnp.mean(train_y), np.max(train_y), np.min(train_y)'

In [100]:
from sklearn.metrics import mean_squared_error, cohen_kappa_score, make_scorer
def reg_scorer(true, pred):
    pred = [min(1, max(0,x)) for x in pred]
    pred = [int(round((x*3)+1)) for x in pred]
    true = [int(round((x*3)+1)) for x in true]
    return cohen_kappa_score(true, pred)

In [101]:
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
#clf = LinearRegression().fit(train_x, train_y)
#clf = SGDRegressor(verbose=1,n_iter_no_change=20).fit(train_x, train_y)
param_grid = {'C': [1,2,5,10], 'epsilon':[0.1,0.2,0.5], 'kernel': ('linear', 'rbf', 'poly','sigmoid')}
svr  = SVR()
scorer = make_scorer(reg_scorer, greater_is_better=True)
clf = GridSearchCV(svr, param_grid, verbose=True,scoring=scorer, n_jobs=-1)
clf.fit(train_x, train_y)
clf.best_estimator_, clf.best_params_, clf.best_score_

Fitting 5 folds for each of 48 candidates, totalling 240 fits


KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Random Forest Regression
random_forest_regression = RandomForestRegressor()

# Parameter Grid
'''random_forest_regression_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}'''
random_forest_regression_param_grid = {
    #'n_estimators': [50, 200],
    #'max_depth': [None, 10, 30],
    #'min_samples_split': [2, 5],
    #'min_samples_leaf': [2, 4],
    'bootstrap': [True, False]
}

rf = GridSearchCV(random_forest_regression, random_forest_regression_param_grid, verbose=True,scoring=scorer, n_jobs=-1, cv=1)
rf.fit(train_x, train_y)
rf.best_estimator_, rf.best_params_, rf.best_score_

In [ ]:
## 0.26 is the best score till now

preds = clf.best_estimator_.predict(dev_x)
mean_squared_error(dev_y, preds),  reg_scorer(dev_y, preds)

preds_rf = rf.best_estimator_.predict(dev_x)
mean_squared_error(dev_y, preds_rf),  reg_scorer(dev_y, preds_rf)

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

svr_best_estimator = clf.best_estimator_
rf_best_estimator = rf.best_estimator_

base_models = [('svr', svr_best_estimator), ('rf', rf_best_estimator)]
meta_model = LinearRegression()  # You can choose a different meta-model if needed

stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model)

stacking_regressor.fit(train_x, train_y)


In [ ]:
test_input =  test.apply(lambda x: x['query']+' '+x['product_title'], axis=1)
test_x = tfidf.transform(test_input)
pred = stacking_regressor.predict(test_x)
pred = [min(1, max(0,x)) for x in pred]
pred = [int(round((x*3)+1)) for x in pred]
out = pd.DataFrame({"id": test.id.to_list(), "prediction": pred})
out.to_csv('submission.csv', index=False)

In [ ]:
'''test_input =  test.apply(lambda x: x['query']+' '+x['product_title'], axis=1)
test_x = tfidf.transform(test_input)
pred = clf.best_estimator_.predict(test_x)
pred = [min(1, max(0,x)) for x in pred]
pred = [int(round((x*3)+1)) for x in pred]
out = pd.DataFrame({"id": test.id.to_list(), "prediction": pred})
out.to_csv('submission.csv', index=False)'''

In [ ]:
'''test_input =  test.apply(lambda x: x['query']+' '+x['product_title'], axis=1)
test_x = tfidf.transform(test_input)
pred = rf.best_estimator_.predict(test_x)
pred = [min(1, max(0,x)) for x in pred]
pred = [int(round((x*3)+1)) for x in pred]
out = pd.DataFrame({"id": test.id.to_list(), "prediction": pred})
out.to_csv('submission.csv', index=False)'''

In [ ]:
sub = pd.read_csv('/kaggle/input/crowdflower-search-relevance/sampleSubmission.csv.zip')
sub

In [ ]:
out